# LangChain: Models, Prompts and Output Parsers
https://learn.deeplearning.ai/langchain/lesson/2/models,-prompts-and-parsers

## Outline

 * Direct API calls to OpenAI API (custom one that enabled by `llama-cpp-python` with Wizard-Vicuna-13B-Uncensored-GGML model at the back)
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

## Get your [OpenAI API Key](https://platform.openai.com/account/api-keys)

In [ ]:
#!pip install python-dotenv
#!pip install openai

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
        max_tokens=1024
    )
    return response.choices[0].message["content"]


In [6]:
get_completion("What is 1+1?")

' The answer to "What is 1+1?" is 2.'

In [7]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [8]:
style = """American English \
in a calm and respectful tone
"""

In [9]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [14]:
response = get_completion(prompt)

In [15]:
response

'I apologize for your unfortunate experience with your blender lid. It sounds like a frustrating situation. Is there anything I can do to assist you in this matter?'

## Chat API : LangChain

Let's try how we can do the same using LangChain.

In [16]:
from langchain.chat_models import ChatOpenAI

In [ ]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature=0.0, max_tokens=1024)
chat

In [20]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [21]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)


In [22]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], output_parser=None, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n', template_format='f-string', validate_template=True)

In [23]:
prompt_template.messages[0].input_variables

['style', 'text']

In [24]:
customer_messages = prompt_template.format_messages(
                    style=style,
                    text=customer_email)

In [25]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain.schema.HumanMessage'>


In [26]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n" additional_kwargs={} example=False


In [27]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)

In [28]:
print(customer_response.content)

I apologize for your unfortunate experience with your blender lid. It sounds like a frustrating situation. Is there anything I can do to assist you in this matter?


In [29]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [30]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [31]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [32]:
service_response = chat(service_messages)
print(service_response.content)

Arrr matey, I be sorry to hear 'bout yer troubles with yer blender. But if ye didn't put the lid on, it be no surprise that it got messy. Aye, we all make mistakes sometimes, but next time, remember ta keep a tight lid on things. Arrrgh!


## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [33]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [34]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [35]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n', template_format='f-string', validate_template=True), additional_kwargs={})]


In [37]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0, max_tokens=1024)
response = chat(messages)
print(response.content)


 The following information can be extracted from the given text:
gift: False
delivery_days: 2
price_value: ["It arrived in two days", "I think my wife liked it so much she was speechless."]


In [ ]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('gift')

### Parse the LLM output string into a Python dictionary

In [39]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [40]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [41]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [42]:
format_instructions = output_parser.get_format_instructions()

In [43]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [44]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [45]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [49]:
%%time
response = chat(messages)

CPU times: user 24.4 ms, sys: 1.58 ms, total: 26 ms
Wall time: 2min 4s


In [50]:
print(response.content) 
# Wizard-Vicuna-13B-Uncensored-GGML model doesn't support output parsers

 Here's an example code snippet that extracts the requested information from the given text:
```python
import re
text = "This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
# Define regular expressions to match the requested information
gift_regex = r"Was the item purchased as a gift for someone else? Answer (True|False) if yes, False if not or unknown.?"
delivery_days_regex = r"How many days did it take for the productto arrive? If this information is not found, output -1."
price_value_regex = r"Extract any sentences about the value or price,and output them as a comma sep

In [ ]:
output_dict = output_parser.parse(response.content)

In [ ]:
output_dict

In [ ]:
type(output_dict)

In [ ]:
output_dict.get('delivery_days')

# LangChain: Memory

## Outline
* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory

## ConversationBufferMemory

In [3]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


In [7]:
llm = ChatOpenAI(temperature=0.0, max_tokens=1024)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory=memory,
    verbose=True
)

In [8]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


' Hello, I am Ava. How can I assist you today?'

In [9]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI:  Hello, I am Ava. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


' The answer to the question "What is 1+1?" is 2.'

In [10]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI:  Hello, I am Ava. How can I assist you today?
Human: What is 1+1?
AI:  The answer to the question "What is 1+1?" is 2.
Human: What is my name?
AI:

> Finished chain.


' Your name has not been provided, so Ava does not know the answer to this question.'

In [11]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI:  Hello, I am Ava. How can I assist you today?
Human: What is 1+1?
AI:  The answer to the question "What is 1+1?" is 2.
Human: What is my name?
AI:  Your name has not been provided, so Ava does not know the answer to this question.


In [12]:
memory.load_memory_variables({})

{'history': 'Human: Hi, my name is Andrew\nAI:  Hello, I am Ava. How can I assist you today?\nHuman: What is 1+1?\nAI:  The answer to the question "What is 1+1?" is 2.\nHuman: What is my name?\nAI:  Your name has not been provided, so Ava does not know the answer to this question.'}

In [13]:
memory = ConversationBufferMemory()

In [14]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [15]:
print(memory.buffer)

Human: Hi
AI: What's up


In [16]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [17]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [18]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

## ConversationBufferWindowMemory

In [19]:
from langchain.memory import ConversationBufferWindowMemory

In [20]:
memory = ConversationBufferWindowMemory(k=1)               

In [21]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [22]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [30]:
llm = ChatOpenAI(temperature=0.0, max_tokens=1024)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [31]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


' Hello, I am Ava. How can I assist you today?'

In [32]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI:  Hello, I am Ava. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


' The answer to the question "What is 1+1?" is 2.'

In [33]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is 1+1?
AI:  The answer to the question "What is 1+1?" is 2.
Human: What is my name?
AI:

> Finished chain.


" I'm sorry, but I do not have access to your name. Please provide more context so that I can assist you better."

## ConversationTokenBufferMemory

In [25]:
#!pip install tiktoken

In [34]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0, max_tokens=1024)

In [35]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [36]:
memory.load_memory_variables({})

{'history': 'AI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

## ConversationSummaryMemory

In [37]:
from langchain.memory import ConversationSummaryBufferMemory


In [38]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [39]:
memory.load_memory_variables({})

{'history': "System:  The human asks what's on the schedule for today, and the AI provides a detailed breakdown of the day's events including meetings, work on a project, lunch with a customer, and more."}

In [40]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [41]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System:  The human asks what's on the schedule for today, and the AI provides a detailed breakdown of the day's events including meetings, work on a project, lunch with a customer, and more.
Human: What would be a good demo to show?
AI:

> Finished chain.


" One possible demonstration could be showing how the AI can analyze data from various sources and provide insights that help businesses make better decisions. For example, it could show how it can identify patterns in customer behavior or predict future trends based on historical data. Another option could be a live demo of the AI's natural language processing capabilities, such as answering questions or performing tasks like booking travel arrangements or ordering food."

In [42]:
memory.load_memory_variables({})

{'history': "System:  The human asks what's on the schedule for today, and the AI provides a detailed breakdown of the day's events including meetings, work on a project, lunch with a customer, and more. The human then asks what would be a good demo to show, and the AI suggests demonstrating the new feature that was added to the project yesterday.\nAI:  One possible demonstration could be showing how the AI can analyze data from various sources and provide insights that help businesses make better decisions. For example, it could show how it can identify patterns in customer behavior or predict future trends based on historical data. Another option could be a live demo of the AI's natural language processing capabilities, such as answering questions or performing tasks like booking travel arrangements or ordering food."}

# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [43]:
import warnings
warnings.filterwarnings('ignore')

In [44]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

## LLMChain

In [45]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [46]:
llm = ChatOpenAI(temperature=0.9, max_tokens=1024)


In [47]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [48]:
chain = LLMChain(llm=llm, prompt=prompt)

In [50]:
product = "Queen Size Sheet Set"
chain.run(product)

' One possible name for such a company could be "Queen Size Linens".'

## SimpleSequentialChain

In [51]:
from langchain.chains import SimpleSequentialChain

In [52]:
llm = ChatOpenAI(temperature=0.9, max_tokens=1024)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [53]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [54]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [55]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
 The best name to describe a company that makes queen size sheet sets would be "bedding manufacturer" or "linens producer".
 Sure, here's your description: "The company specializes in producing high-quality queen size sheets and bedding essentials like pillowcases, comforters, and duvet covers."

> Finished chain.


' Sure, here\'s your description: "The company specializes in producing high-quality queen size sheets and bedding essentials like pillowcases, comforters, and duvet covers."'

## SequentialChain

In [56]:
from langchain.chains import SequentialChain

In [57]:
llm = ChatOpenAI(temperature=0.9, max_tokens=1024)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [58]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [62]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [59]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [63]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [64]:
review = "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?"
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': '"I find the taste mediocre. The foam doesn\'t hold, it\'s strange. I buy the same in the store and the taste is better..."',
 'summary': ' "The quality of the coffee sold in the store is better than the one sampled at the café."',
 'followup_message': ' "Le café vendu dans la boutique a une meilleure qualité que celui essayé au café."'}

## Router Chain

In [65]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [66]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [67]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

In [68]:
llm = ChatOpenAI(temperature=0.0, max_tokens=1024)

In [69]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [70]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [71]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [72]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [73]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [74]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...


OutputParserException: Parsing text
 To answer your question about black body radiation, we recommend using the "physics" prompt. However, if you revise your input to something like "Can you explain how black body radiation works?" or "What is the relationship between temperature and black body radiation?", it may lead to a better response from the language model.
 raised following error:
Got invalid JSON object. Error: Expecting value: line 1 column 1 (char 0)

In [ ]:
chain.run("what is 2 + 2")

In [ ]:
chain.run("Why does every cell in our body contain DNA?")

# LangChain: Agents

## Outline:

* Using built in LangChain tools: DuckDuckGo search and Wikipedia
* Defining your own tools

In [80]:
%pip install -U wikipedia

20858.18s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 3.3 MB/s eta 0:00:00a 0:00:01
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=1be3bc73a7cfe27b9e7a66d4f304a4258f82e3446267ea614d4a0ad6d74da1ef
  Stored in directory: /home/limcheekin/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia
Note: you may need to restart the kernel to use updated packages.


In [86]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

In [87]:
llm = ChatOpenAI(temperature=0, max_tokens=1024)

In [88]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

In [89]:
agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [90]:
# agent("What is the 25% of 300?") doesn't works!
agent("Calculate the value of 25% of 300.")




> Entering new AgentExecutor chain...


Could not parse LLM output:  {
 "action": "Calculator",
 "action_input": "What is 25% of 300?"
}

### 
Observation: Invalid or incomplete response
Thought: I apologize for the confusion. Here's a valid $JSON_BLOB that uses the Calculator tool to calculate 25% of 300:
```
{
 "action": "Calculator",
 "action_input": "What is 25% of 300?"
}
```
Action:
```
$JSON_BLOB
```


ValueError: unknown format from LLM: To calculate 25% of a number, we need to find out what percentage it is and then multiply by 0.25. In this case, 25% of 300 is (0.25 * 300) = 75.
```text
25% of 300
```
...numexpr.evaluate("(0.25 * 300)")...

In [91]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
result = agent(question) 



> Entering new AgentExecutor chain...
Could not parse LLM output: Here's your $JSON_BLOB:
```
{
 "action": Wikipedia,
 "action_input": Tom M. Mitchell CMU
}
```

And here's the result of running that action:
```
Question: What book did Tom M. Mitchell write?
Action: 
```
$JSON_BLOB
```

Observation: Invalid or incomplete response
Thought:Could not parse LLM output:  The book is called "Machine Learning: A Guide to the Theory and Practice of Learning Machines".
Observation: Invalid or incomplete response
Thought:Final Answer: Machine Learning: A Guide to the Theory and Practice of Learning Machines

> Finished chain.


## Python Agent

In [92]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [93]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [94]:
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 



> Entering new AgentExecutor chain...


OutputParserException: Could not parse LLM output: ` I think we should use the `sorted()` function to sort the list by last name and then first name. We can also print the output using a for loop.`

#### View detailed outputs of the chains

In [95]:
import langchain
langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 
langchain.debug=False

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nI

OutputParserException: Could not parse LLM output: ` I think we should use the `sorted()` function to sort the list by last name and then first name. We can also print the output using a for loop.`

## Define your own tool

In [96]:
#!pip install DateTime

In [97]:
from langchain.agents import tool
from datetime import date

In [98]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [99]:
agent= initialize_agent(
    tools + [time], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

**Note**: 

The agent will sometimes come to the wrong conclusion (agents are a work in progress!). 

If it does, please try running it again.

In [101]:
try:
    result = agent("whats the date today?") 
except: 
    print("exception on external access")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "whats the date today?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "whats the date today?",
  "agent_scratchpad": "",
  "stop": [
    "Observation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Answer the following questions as best you can. You have access to the following tools:\n\nCalculator: Useful for when you need to answer questions about math.\nWikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\ntime: time(text: str) -> str - Returns todays date, use this for any     questions related to knowing todays date.     The input should always be an empty string,     and this function will always return to